In [3]:
import numpy as np
import rasterio
from PIL import Image

In [7]:
def calculate_ndvi(b4_file, b8_file, output_file, tile_size=1024):
    red_src = Image.open(b4_file)
    nir_src = Image.open(b8_file)

    width, height = red_src.size

    ndvi_image = np.zeros((height, width), dtype=np.uint8)

    for i in range(0, height, tile_size):
        for j in range(0, width, tile_size):
            # Read the Red band image (Band 4)
            b4_data = np.array(
                red_src.crop(
                    (j, i, min(j + tile_size, width), min(i + tile_size, height))
                )
            )
            # Read the NIR band image (Band 8)
            b8_data = np.array(
                nir_src.crop(
                    (j, i, min(j + tile_size, width), min(i + tile_size, height))
                )
            )

            # Avoid division by zero by replacing 0 values in the denominator with a small number
            denominator = b8_data + b4_data
            denominator[denominator == 0] = (
                1e-10  # Small value to avoid division by zero
            )

            # Calculate NDVI using the formula (B8 - B4) / (B8 + B4)
            ndvi = (b8_data - b4_data) / denominator
            print(ndvi)
            # Normalize NDVI values to the 0-255 range for saving as PNG
            ndvi_normalized = np.clip((ndvi + 1) * 127.5, 0, 255).astype(np.uint8)

            # Place the tile in the output image
            ndvi_image[
                i : min(i + tile_size, height), j : min(j + tile_size, width)
            ] = ndvi_normalized

    # Convert the numpy array to a PIL image
    ndvi_image = Image.fromarray(ndvi_image)

    # Save the NDVI image as a PNG
    ndvi_image.save(output_file)

    print(f"NDVI image saved to {output_file}")


# Example usage:
b4_file = "2024-11-04-00_00_2024-11-04-23_59_Sentinel-2_L2A_B04_(Raw).jpg" 
b8_file = "2024-11-04-00_00_2024-11-04-23_59_Sentinel-2_L2A_B08_(Raw).jpg"
output_file = "combined_output.png"  # Output PNG file path
calculate_ndvi(b4_file, b8_file, output_file)

C:\Users\joshu\AppData\Local\Temp\ipykernel_25544\1878982871.py:31: RuntimeWarning: divide by zero encountered in divide
  ndvi = (b8_data - b4_data) / denominator
C:\Users\joshu\AppData\Local\Temp\ipykernel_25544\1878982871.py:31: RuntimeWarning: invalid value encountered in divide
  ndvi = (b8_data - b4_data) / denominator
C:\Users\joshu\AppData\Local\Temp\ipykernel_25544\1878982871.py:34: RuntimeWarning: invalid value encountered in cast
  ndvi_normalized = np.clip((ndvi + 1) * 127.5, 0, 255).astype(np.uint8)


[[0.01960784 0.02380952 0.024      ... 0.0867052  0.08536585 0.13414634]
 [2.         7.         7.         ... 0.11111111 0.11392405 0.17283951]
 [1.66666667 1.8        1.33333333 ... 0.11801242 0.1257485  0.16201117]
 ...
 [7.         0.1755102  0.11587983 ... 0.14666667 0.1503268  0.15384615]
 [0.11372549 0.14754098 0.10084034 ... 0.13548387 0.14285714 0.14285714]
 [0.16216216 0.2        0.12888889 ... 0.12       0.12258065 0.12345679]]
[[0.12345679 0.15555556 0.14130435 ... 0.06666667 0.04932735 0.03349282]
 [0.13793103 0.15183246 0.13989637 ... 0.04166667 0.03061224 0.02590674]
 [0.11917098 0.12380952 0.10900474 ... 0.08074534 0.09756098 0.10714286]
 ...
 [0.17073171 0.12432432 0.0880829  ... 0.19587629 0.27710843 0.4025974 ]
 [0.14619883 0.12903226 0.09574468 ... 0.31034483 0.35       0.42168675]
 [0.11111111 0.12359551 0.11363636 ... 0.31343284 0.30434783 0.39240506]]
[[0.04040404 0.05882353 0.07777778 ... 0.06586826 0.05882353 0.05952381]
 [0.06285714 0.0877193  0.10344828 ... 